#### Lesson 2: Sequential Chats and Customer Onboarding

#### Setup

In [1]:
#r "nuget:AutoGen,0.1.0"
#load "util.csx"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using System.Threading;

var openAIModel = "gpt-4o-mini";
var openaiClient = OpenAIClientProvider.Create();

Installed Packages AutoGen, 0.1.0

Create the needed agents

In [2]:
var onboardingPersonalInformationAgent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Onboarding_Personal_Information_Agent",
    modelName: openAIModel,
    systemMessage: """
    You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [3]:
var onboardingTopicPreferenceAgent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Onboarding_Topic_Preference_Agent",
    modelName: openAIModel,
    systemMessage: """
    You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [4]:
var customerEngagementAgent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Customer_Engagement_Agent",
    modelName: openAIModel,
    systemMessage: """
    You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [5]:
var summarizer = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "Summarizer",
    modelName: openAIModel,
    systemMessage: """
    You are a helpful summarizer agent.
    Your job is to summarize the conversation between the user and the customer service agent.
    Return 'TERMINATE' when you are done.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [6]:
var user = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "User",
    modelName: openAIModel,
    systemMessage: """
    Your name is John and you live in New York.
    You are reaching out to customer service to find out something fun.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

#### Creating Tasks
In python AutoGen, you can use initiate_chats to create and run a sequential of tasks in json object
In dotnet AutoGen, however, that feature is not available, so you need to manually create these tasks using code.

#### Task 1. Onboard customer by gathering name and location
(onboard_personal_information_agent -> user .. (repeat less than two times)) -> summarizer

In [7]:
var greetingMessage = new TextMessage(Role.Assistant, """
    Hello, I'm here to help you get started with our product.
    Could you tell me your name and location?
    """, from: onboardingPersonalInformationAgent.Name);

var conversation = await onboardingPersonalInformationAgent.SendAsync(
    receiver: user,
    [greetingMessage],
    maxRound: 2)
    .ToListAsync();

var summarizePrompt = """
    Return the customer information into as JSON object only: {'name': '', 'location': ''}
    """;

var summary = await summarizer.SendAsync(summarizePrompt, conversation);

from: User
Hi! My name is John and I live in New York. I'm excited to get started!

from: Onboarding_Personal_Information_Agent
Great to meet you, John! Thank you for sharing your location. 

TERMINATE

from: Summarizer
```json
{
  "name": "John",
  "location": "New York"
}
```



#### Task 2. Gapther customer's preferences on news topics
(onboarding_topic_preference_agent -> user .. (repeat one time)) -> summarizer

In [8]:
var topicPreferenceMessage = new TextMessage(Role.Assistant, """
    Great! Could you tell me what topics you are interested in reading about?
    """, from: onboardingTopicPreferenceAgent.Name);

conversation = await onboardingTopicPreferenceAgent.SendAsync(
    receiver: user,
    [topicPreferenceMessage],
    maxRound: 1)
    .ToListAsync();

// Keep summarizing
summary = await summarizer.SendAsync(chatHistory: new[] { summary }.Concat(conversation));

from: User
Hi there! I’m really interested in fun topics related to travel, culinary experiences, and unique events happening around the world. Anything quirky or unusual would definitely catch my attention! What do you have in mind?

from: Summarizer
The user is interested in fun topics related to travel, culinary experiences, and unique events around the world, specifically looking for quirky or unusual content. TERMINATE



#### Task 3. Engage the customer with fun facts, jokes, or interesting stories based on the user's personal information and topic preferences
(user(find fun thing to read) -> customerEngagementAgent .. (repeat 1 time)) -> summarizer

In [9]:
var funFactMessage = new TextMessage(Role.User, """
    Let's find something fun to read.
    """, from: user.Name);

conversation = await user.SendAsync(
    receiver: customerEngagementAgent,
    chatHistory: conversation.Concat([funFactMessage]), // this time, we keep the previous conversation history
    maxRound: 1)
    .ToListAsync();

// Keep summarizing
summary = await summarizer.SendAsync(chatHistory: new[] { summary }.Concat(conversation));

from: Customer_Engagement_Agent
Absolutely, let’s embark on a fun journey together!

### Quirky Travel Facts:
1. **The Town of Dull**: There's a village in Scotland called Dull, and it has a sister town in the USA named Boring, Oregon. Together, they form the ultimate “Dull and Boring” travel destination!

2. **The Great Wall of China**: While it’s iconic, many people don’t know that the Great Wall is not a single continuous wall. It’s actually a series of walls and fortifications built over centuries. It’s so long that it could wrap around the Earth nearly 1/2 a time!

### Culinary Curiosities:
- **Fried Spider, Anyone?**: In Cambodia, fried tarantulas are a delicacy! They’re crispy on the outside and often served with garlic and lime. It’s a unique taste that adventurous eaters say is similar to soft shell crab!

- **Chocolate-Covered Insects**: If you’re looking for something sweet and crunchy, how about trying chocolate-covered crickets? They’re packed with protein and are a popula